In [1]:
import numpy as np

# Player

Here is the player class. This class has all the requirements that homework needed. For extra, I implemented the str method and the repr method of the class so that I can print the objects individually also in a list. I also implemented == method for the class because I needed in the functions I used.

Printing structure => {player name}--{peerWinner, true or false} -> {rank}

In [2]:
class Player:
    def __init__(self, name : str, rank : int, peerWinner : bool):
        self.name : str = name
        self.rank : int = rank
        self.peerWinner : bool = peerWinner

    def __eq__(self, other):
        return self.name == other.name
    
    def getrank(self) -> int:
        return self.rank
    
    def setrank(self, rank : int) -> None:
        self.rank = rank

    def __str__(self):
        return f"{self.name}--{self.peerWinner} -> {self.rank}"
    
    def __repr__(self):
        return str(self)



# Functions That Are Needed For the Algorithm

These functions are the same functions that are provided in the week5 notebook. However, that notebook is written with a data oriented programming logic. I changed it into a object oriented programming convention. 

## Programming Logic
Since I can have the all properties inside the Player class, I am not using dictionaries to keep track of the ranks, names, and peerWinner properties. Instead, I use only a list of Players, and I do all my calculations on this list.

## Changes

Besides changing the paradigm of the code, no algorithm is changed on these functions, they all working same as they were working with dictionaries.

## Assertions

I am writing my code using python3's type checking feature. So, I didn't need assertions most of the time since I was already checking the types with most of the functions, but of course, I added all the assertions that I detected.

In [3]:
def enumeration(P):
    lst = np.arange(len(P))
    order = np.random.permutation(lst)
    return [P[ind] for ind in order]

def rankeds(P, r):
    targets = filter(lambda p : p.rank == r, P)
    return list(targets)


def ladder_match(P : list[Player], p : Player, q : Player):
    m = match(p, q, 0.5)
    if m is None : return P

    if m == q:
      if q.getrank() == None:
        q.setrank(p.getrank())
        return

      tempRank : int = p.getrank()


      p.setrank(q.getrank())
      q.setrank(tempRank)
      del tempRank

    
def match(p : Player, q : Player, thr : float, tieflag : bool =True):
    toss = np.random.rand(1)
    upC = 1
    downC = 1
    if tieflag: # If Tie is allowed
      upC = 1.1
      downC = 0.9
    if thr * downC < toss < thr * 1.1: # If the random number is around thr, result is a tie
        result = None # Tie
    elif toss > thr * upC:
        result = p
    else:
        result = q
    return result

def unrankedplayers(n):
    P = []
    for ind in range(n):
        p = Player(f'P{ind}', None, True)
        P.append(p)
    return P


def initialrankadjustment(P : list[Player], S):
    assert len(P) == sum(S)
    M = enumeration(P)
    m = len(S)
    R = P.copy()
    R = enumeration(R)

    for ind in range(S[0]):
        R[ind].rank = 0 # The initial champion(s)
    c =  S[0]

    for rank in range(1, m):
        W = rankeds(R, rank - 1) # The runners-up
        Mp = enumeration(W)

        for ind in range(S[rank]):

            R[c + ind].setrank(rank)
            jnd = ind % len(Mp)

            if Mp[jnd].rank != rank:
                ladder_match(R, Mp[jnd], M[c + ind])
        c += S[rank]
    return R

def findPlayer(P : list[Player], name : str):
   for i in P:
      if i.name == name : 
         return i

def pyramidmatch(P, p, q):
  
  pi = findPlayer(P,p.name)
  qi = findPlayer(P,q.name)


  rank_p = pi.getrank()
  peerWinner_p = pi.peerWinner
  rank_q = qi.getrank()
  peerWinner_q = qi.getrank()


  if not ((rank_p == rank_q) and (not peerWinner_q)) | ((rank_p == rank_q - 1) and peerWinner_q):
    return P
  
  R = P.copy() # We copy the player dictionary first
  R.remove(pi)
  R.remove(qi)
  

  # We do not have the two players in our set of players now.
  m = match(p, q, 0.5) # Let them compete
  
  if (rank_p == rank_q): # Peer match
    if (((m == p) and (not peerWinner_p)) or (((m == q) or (m is None)) and peerWinner_p)):
      pp = p
      rank_pp = rank_p
      peerWinner_pp = (m == p)
    else:
      pp = p
      rank_pp = rank_p
      peerWinner_pp = peerWinner_p
    if (m == q):
      qp = q
      rank_qp = rank_q
      peerWinner_qp = True
    else:
      qp = q
      rank_qp = rank_q
      peerWinner_qp = peerWinner_q

    R.append(Player('P'+str(pp),rank_pp,peerWinner_pp))
    R.append(Player('P'+str(qp),rank_qp,peerWinner_qp))
    return R
  
  else: # Rank challenge match
    qp = q
    rank_qp = rank_q
    peerWinner_qp = False
    if (m == p) or m is None:
      R.append(Player('P'+str(p),rank_p,peerWinner_p))
      R.append(Player('P'+str(qp),rank_qp,peerWinner_qp))
      return R
    else:
      pp = p
      peerWinner_pp = False
      rank_pp = rank_q
      rank_qp = rank_p 

      R.append(Player('P'+str(pp),rank_pp,peerWinner_pp))
      R.append(Player('P'+str(qp),rank_qp,peerWinner_qp))
      return R

In [4]:
# This code is written by following the pseudo code in the textbook. No logic is different.

def kingofthehill(P : set):
    n = len(P)
    assert 1 <= n 
    assert ((n+1) & (n)) == 0 # 2^n - 1  check

    m = int(np.log2(n + 1))

    S = [2**i for i in range(m)]
    R = initialrankadjustment(P,S)

    for rprime in range(m):
        r = (m - 1) - (rprime - 1)
        M = enumeration(rankeds(R,r))
        l = len(M)

        for i in range(int(l//2-1)):
            M1 = M[2 * i]
            M2 = M[2 * i + 1]
            M1.peerWinner = False
            M2.peerWinner = False

            R = pyramidmatch(R,M1,M2)
        templist = rankeds(R,r)
        targets = filter(lambda p : p.peerWinner, templist)
        M = list(targets)
        Mprime = enumeration(rankeds(R, r-1))

        for i in range(int(len(M)//2 -1)):
            print(Mprime,M, i)
            R = pyramidmatch(R, Mprime[i], M[i])
            
    return R

## Testing

I am testing my code down here, used n as 32 - 1 = 31. 

In [5]:

P = unrankedplayers(31)


K = kingofthehill(P)
from pprint import pprint # This is only for pretty printing, nothing else
pprint(K)

[P3--True -> 1,
 P2--True -> 0,
 P25--True -> 1,
 P11--False -> 3,
 P21--False -> 3,
 P4--False -> 2,
 P7--True -> 3,
 P13--True -> 2,
 P28--False -> 3,
 P19--False -> 3,
 P26--True -> 2,
 P1--False -> 3,
 P0--False -> 2,
 P12--False -> 3,
 P29--True -> 3,
 P9--False -> 4,
 P8--True -> 4,
 P17--False -> 4,
 P15--False -> 4,
 P23--False -> 4,
 P30--False -> 4,
 P16--False -> 4,
 P6--False -> 4,
 P5--False -> 4,
 P27--False -> 4,
 P24--False -> 4,
 P18--True -> 4,
 P22--False -> 4,
 P10--False -> 4,
 P20--False -> 4,
 P14--False -> 4]
